# SageMaker Example

## 2. Build the container

demo codes are in `app/`
build and push the docker with following commands:

In [5]:
!pip install  -U sagemaker boto3

In [ ]:
!bash build_and_push_sglang.sh

## 3. Deploy on SageMaker

define the model and deploy on SageMaker


### 3.1 Init SageMaker session

In [64]:
# !pip install boto3 sagemaker transformers
import re
import json
import os,dotenv
import boto3
import sagemaker
from sagemaker import Model


dotenv.load_dotenv()
print(os.environ)

boto_sess = boto3.Session(
    region_name='us-east-1'
)

sess = sagemaker.session.Session(boto_session=boto_sess)
# role = sagemaker.get_execution_role()
role = os.environ.get('role')

environ({'USER': 'ubuntu', 'SSH_CLIENT': '52.94.133.139 4026 22', 'XDG_SESSION_TYPE': 'tty', 'SHLVL': '2', 'HOME': '/home/ubuntu', 'SSL_CERT_FILE': '/usr/lib/ssl/cert.pem', 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1000/bus', 'LOGNAME': 'ubuntu', '_': '/home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/bin/python', 'XDG_SESSION_CLASS': 'user', 'XDG_SESSION_ID': '34414', 'VSCODE_CLI_REQUIRE_TOKEN': '3ae02bac-b0cf-4129-8cc1-92a260fb9929', 'PATH': '/home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/bin:/home/ubuntu/.vscode-server/cli/servers/Stable-e54c774e0add60467559eb0d1e229c6452cf8447/server/bin/remote-cli:/home/ubuntu/.local/bin:/home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/bin:/home/ubuntu/workspace/llm_model_hub/miniconda3/condabin:/home/ubuntu/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin', 'VSCODE_AGENT_FOLDER': '/home/ubuntu/.vscode-server', 'XDG_RUNTIME_DIR': '/run/user/1000', 'S

[03/01/25 11:29:26] INFO     Found credentials from IAM Role: admin_role_for_workshop           ]8;id=243716;file:///home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/lib/python3.11/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=529584;file:///home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/lib/python3.11/site-packages/botocore/credentials.py#1132\1132]8;;\

In [ ]:
# role

### 3.2 Prepare model file

#### Option 2: deploy vllm by model_id

In [ ]:
!tar czvf model.tar.gz model_tar/

In [65]:


s3_code_prefix = f"sagemaker_endpoint/sglang/"
bucket = sess.default_bucket() 
code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-1-434444145045/sagemaker_endpoint/sglang//model.tar.gz


### 3.3 Deploy model

### test deployment from s3

In [66]:
import sagemaker
import boto3
sm_client = boto3.client(service_name="sagemaker")


### Create a SageMaker model



(llama3-8b-scale-to-zero-autoscaling)[https://github.com/aws-samples/sagemaker-genai-hosting-examples/blob/main/scale-to-zero-endpoint/llama3-8b-scale-to-zero-autoscaling.ipynb]

In [67]:

CONTAINER='434444145045.dkr.ecr.us-east-1.amazonaws.com/sagemaker_endpoint/sglang:v0.4.3.post2-cu124'
# CONTAINER='434444145045.dkr.ecr.us-east-1.amazonaws.com/sagemaker_endpoint/vllm:v0.7.2'
model_path = "s3://sagemaker-us-east-1-434444145045/Qwen2-1-5B-Instruct/6d0410c634ea438fa5018072e84c10a6/finetuned_model_merged/"
model_id = 'Qwen/Qwen2-1.5B-Instruct'
base_name = sagemaker.utils.name_from_base("sagemaker")
sm_client = boto3.client(service_name="sagemaker")
model_name = base_name +"-model"
endpoint_name = base_name+"-endpoint"
component_name = base_name+"-component"
endpoint_config_name =  base_name+"-config"
env={
    "HF_MODEL_ID": model_id,
    "S3_MODEL_PATH":model_path,
}
container_config = {
    'Image': CONTAINER,
    'ModelDataUrl': code_artifact,
    'Environment': env
}

print(model_name)
print(endpoint_name)
print(component_name)
print(endpoint_config_name)

sagemaker-2025-03-01-11-30-13-897-model
sagemaker-2025-03-01-11-30-13-897-endpoint
sagemaker-2025-03-01-11-30-13-897-component
sagemaker-2025-03-01-11-30-13-897-config


In [76]:
# endpoint_name = "sagemaker-2025-03-01-11-30-13-897-endpoint"
# component_name = "sagemaker-2025-03-01-11-30-13-897-component"

In [68]:

response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer=container_config
)

print(f"Model created: {response['ModelArn']}")

Model created: arn:aws:sagemaker:us-east-1:434444145045:model/sagemaker-2025-03-01-11-30-13-897-model


### Create a SageMaker endpoint configuration

In [69]:
instance_type = "ml.g5.2xlarge"
model_data_download_timeout_in_seconds = 1200
container_startup_health_check_timeout_in_seconds = 1200

min_instance_count = 0 # Minimum instance must be set to 0
max_instance_count = 2

sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ExecutionRoleArn=role,
    ProductionVariants=[
        {
            "VariantName": "AllTraffic",
            "InstanceType": instance_type,
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": model_data_download_timeout_in_seconds,
            "ContainerStartupHealthCheckTimeoutInSeconds": container_startup_health_check_timeout_in_seconds,
            "ManagedInstanceScaling": {
                "Status": "ENABLED",
                "MinInstanceCount": min_instance_count,
                "MaxInstanceCount": max_instance_count,
            },
            "RoutingConfig": {"RoutingStrategy": "LEAST_OUTSTANDING_REQUESTS"},
        }
    ],
)

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:434444145045:endpoint-config/sagemaker-2025-03-01-11-30-13-897-config',
 'ResponseMetadata': {'RequestId': '7cb304a6-9f48-40df-83be-c1280aeebb2b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7cb304a6-9f48-40df-83be-c1280aeebb2b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '121',
   'date': 'Sat, 01 Mar 2025 11:30:46 GMT'},
  'RetryAttempts': 0}}

### Create the SageMaker endpoint



In [70]:
sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)

{'EndpointArn': 'arn:aws:sagemaker:us-east-1:434444145045:endpoint/sagemaker-2025-03-01-11-30-13-897-endpoint',
 'ResponseMetadata': {'RequestId': '327a097c-60aa-4f21-b799-5df311ab327e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '327a097c-60aa-4f21-b799-5df311ab327e',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '110',
   'date': 'Sat, 01 Mar 2025 11:46:29 GMT'},
  'RetryAttempts': 0}}

In [71]:
import time
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating


Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-1:434444145045:endpoint/sagemaker-2025-03-01-11-30-13-897-endpoint
Status: InService


In [72]:
t1 = time.time()
sm_client.create_inference_component(
    InferenceComponentName=component_name,
    EndpointName=endpoint_name,
    VariantName="AllTraffic",
    Specification={
        "ModelName": model_name,
        "ComputeResourceRequirements": {
		    "NumberOfAcceleratorDevicesRequired": 1, 
			#"NumberOfCpuCoresRequired": 2, 
			"MinMemoryRequiredInMb": 1024*8
	    }
    },
    RuntimeConfig={"CopyCount": 1},
)

{'InferenceComponentArn': 'arn:aws:sagemaker:us-east-1:434444145045:inference-component/sagemaker-2025-03-01-11-30-13-897-component',
 'ResponseMetadata': {'RequestId': 'ca463816-e42f-47ad-8328-6483a6a7ad9e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ca463816-e42f-47ad-8328-6483a6a7ad9e',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '132',
   'date': 'Sat, 01 Mar 2025 11:48:55 GMT'},
  'RetryAttempts': 0}}

## 4. Test

you can invoke your model with SageMaker SDK

### 4.1 Message api non-stream mode

In [97]:
def check_inference_component_status(inference_component_name):
    sagemaker_client = boto3.client('sagemaker')
    
    try:
        response = sagemaker_client.describe_inference_component(
            InferenceComponentName=inference_component_name
        )
        # print(response)
        status = response['InferenceComponentStatus']
        print(f"Inference Component Status: {status}")
        
        if status == 'InService':
            print("Inference component has been successfully deployed")
            return True
        elif status in ['Creating', 'Updating']:
            print("Inference component is still being deployed...")
            return False
        else:
            print(f"Inference component deployment failed with status: {status}")
            # You might want to check the FailureReason if available
            if 'FailureReason' in response:
                print(f"Failure reason: {response['FailureReason']}")
            return True
            
    except Exception as e:
        print(f"Error checking inference component status: {str(e)}")
        return True
    
while True:
    if  check_inference_component_status(component_name):
        print(f"Inference Component is ready:{time.time()-t1:.1f} s")
        break
    time.sleep(30)
    
from sagemaker.predictor import retrieve_default 
from sagemaker import Predictor
from sagemaker import serializers, deserializers
# predictor = retrieve_default(endpoint_name, sagemaker_session=sess) 

predictor = Predictor(
            endpoint_name=endpoint_name,
            sagemaker_session=sess,
            serializer=serializers.JSONSerializer(),
            component_name=component_name
        )

payload = {
    "messages": [
    {
        "role": "user",
        "content": "who are you"
    }
    ],
    "model":"qwen",
    "max_tokens": 1024,
    "stream": False
}
response = predictor.predict(payload) 
print(response)

Inference Component Status: InService
Inference component has been successfully deployed
Inference Component is ready:1991.7 s
b'{"id":"eea20689d118484b9d512b333fa44fe9","object":"chat.completion","created":1740832513,"model":"qwen","choices":[{"index":0,"message":{"role":"assistant","content":"I am an artificial intelligence designed to assist with answering questions, providing information, and engaging in conversation. I am here to help you with any queries you may have.","tool_calls":null},"logprobs":null,"finish_reason":"stop","matched_stop":151645}],"usage":{"prompt_tokens":22,"total_tokens":56,"completion_tokens":34,"prompt_tokens_details":null}}'


In [77]:
# component_name = "sagemaker-2025-03-01-00-39-49-809-component"
# endpoint_name = "sagemaker-2025-03-01-00-39-49-809-endpoint"
runtime = boto3.client('runtime.sagemaker',region_name='us-east-1')
payload = {
    "messages": [
    {
        "role": "user",
        "content": "who are you"
    }
    ],
    "model":"qwen",
    "max_tokens": 1024,
    "stream": False
}
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    InferenceComponentName=component_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

print(json.loads(response['Body'].read())["choices"][0]["message"]["content"])

I am a large language model, created by Alibaba Cloud. I am designed to answer questions and provide assistance to users. I can understand and respond in over 100 languages.


### 4.2 Message api stream mode

In [78]:
payload = {
    "messages": [
    {
        "role": "user",
        "content": "Write a quick sort in python"
    }
    ],
    "model":"custome",
    "max_tokens": 4096,
    "stream": True
}

response = runtime.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    InferenceComponentName=component_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

buffer = ""
for t in response['Body']:
    buffer += t["PayloadPart"]["Bytes"].decode()
    last_idx = 0
    for match in re.finditer(r'^data:\s*(.+?)(\n\n)', buffer):
        try:
            data = json.loads(match.group(1).strip())
            last_idx = match.span()[1]
            print(data["choices"][0]["delta"]["content"], end="")
        except (json.JSONDecodeError, KeyError, IndexError) as e:
            pass
    buffer = buffer[last_idx:]

Sure, here's a simple Python implementation of the quicksort algorithm:

```python
def quicksort(arr):
    if len(arr) <= 1:
        return arr

    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]

    return quicksort(left) + middle + quicksort(right)
```

To use the function, you can call it and pass in the list of elements you want to sort:

```python
my_list = [3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5]
sorted_list = quicksort(my_list)
print(sorted_list)
```

This will output: `[1, 1, 2, 3, 3, 4, 5, 5, 5, 6, 9]` which is the sorted version of the original list.

### 4.3 Register a new autoscaling target
After you create your SageMaker endpoint and inference components, you register a new auto scaling target for Application Auto Scaling. In the following code block, you set MinCapacity to 0, which is required for your endpoint to scale down to zero

https://aws.amazon.com/cn/blogs/machine-learning/unlock-cost-savings-with-the-new-scale-down-to-zero-feature-in-amazon-sagemaker-inference/

In [79]:
aas_client = sess.boto_session.client("application-autoscaling")
cloudwatch_client = sess.boto_session.client("cloudwatch")

- 这一步必须，先注册resource_id

In [80]:

# Autoscaling parameters
resource_id = f"inference-component/{component_name}"
service_namespace = "sagemaker"
scalable_dimension = "sagemaker:inference-component:DesiredCopyCount"

min_copy_count = 0
max_copy_count = 4

aas_client.register_scalable_target(
    ServiceNamespace=service_namespace,
    ResourceId=resource_id,
    ScalableDimension=scalable_dimension,
    MinCapacity=min_copy_count,
    MaxCapacity=max_copy_count,
)

{'ScalableTargetARN': 'arn:aws:application-autoscaling:us-east-1:434444145045:scalable-target/056mbae2deb9e21a45c0bd3cf073cece1bd5',
 'ResponseMetadata': {'RequestId': '9d3310db-9460-4100-8d42-b5a1db129616',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9d3310db-9460-4100-8d42-b5a1db129616',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '131',
   'date': 'Sat, 01 Mar 2025 11:56:17 GMT'},
  'RetryAttempts': 0}}

- After you have registered your new scalable target, the next step is to define your target tracking policy. In the following code example, we set the TargetValue to 5. This setting instructs the auto scaling system to increase capacity when the number of concurrent requests per model reaches or exceeds 5.

In [81]:
aas_client.describe_scalable_targets(
    ServiceNamespace=service_namespace,
    ResourceIds=[resource_id],
    ScalableDimension=scalable_dimension,
)

# The policy name for the target traking policy
target_tracking_policy_name = f"Target-tracking-policy-{component_name}"

aas_client.put_scaling_policy(
    PolicyName=target_tracking_policy_name,
    PolicyType="TargetTrackingScaling",
    ServiceNamespace=service_namespace,
    ResourceId=resource_id,
    ScalableDimension=scalable_dimension,
    TargetTrackingScalingPolicyConfiguration={
        "PredefinedMetricSpecification": {
            "PredefinedMetricType": "SageMakerInferenceComponentConcurrentRequestsPerCopyHighResolution",
        },
        # Low TPS + load TPS
        "TargetValue": 5,  # you need to adjust this value based on your use case
        "ScaleInCooldown": 180,  # default 300
        "ScaleOutCooldown": 180,  # default 300
    },
)

{'PolicyARN': 'arn:aws:autoscaling:us-east-1:434444145045:scalingPolicy:bae2deb9-e21a-45c0-bd3c-f073cece1bd5:resource/sagemaker/inference-component/sagemaker-2025-03-01-11-30-13-897-component:policyName/Target-tracking-policy-sagemaker-2025-03-01-11-30-13-897-component',
 'Alarms': [{'AlarmName': 'TargetTracking-inference-component/sagemaker-2025-03-01-11-30-13-897-component-AlarmHigh-93acb60a-b192-4074-91ad-8d423c230d78',
   'AlarmARN': 'arn:aws:cloudwatch:us-east-1:434444145045:alarm:TargetTracking-inference-component/sagemaker-2025-03-01-11-30-13-897-component-AlarmHigh-93acb60a-b192-4074-91ad-8d423c230d78'},
  {'AlarmName': 'TargetTracking-inference-component/sagemaker-2025-03-01-11-30-13-897-component-AlarmLow-8626488f-9814-496e-b1db-25e45ffa3c3a',
   'AlarmARN': 'arn:aws:cloudwatch:us-east-1:434444145045:alarm:TargetTracking-inference-component/sagemaker-2025-03-01-11-30-13-897-component-AlarmLow-8626488f-9814-496e-b1db-25e45ffa3c3a'}],
 'ResponseMetadata': {'RequestId': '8394508

### Scale out from zero policy (step scaling policy )
To enable your endpoint to scale out from zero instances, do the following:

Configure Step Scaling Policy
Create a step scaling policy that defines when and how to scale out from zero. This policy will add 1 model copy when triggered, enabling SageMaker to provision the instances required to handle incoming requests after being idle. The following shows you how to define a step scaling policy. Here we have configured to scale out from 0 to 1 model copy ("ScalingAdjustment": 1), depending on your use case you can adjust ScalingAdjustment as required.


In [82]:
#The policy name for the step scaling policy

# Autoscaling parameters
resource_id = f"inference-component/{component_name}"
service_namespace = "sagemaker"
scalable_dimension = "sagemaker:inference-component:DesiredCopyCount"
step_scaling_policy_name = f"Step-scaling-policy-{component_name}"

aas_client.put_scaling_policy(
    PolicyName=step_scaling_policy_name,
    PolicyType="StepScaling",
    ServiceNamespace=service_namespace,
    ResourceId=resource_id,
    ScalableDimension=scalable_dimension,
    StepScalingPolicyConfiguration={
        "AdjustmentType": "ChangeInCapacity",
        "MetricAggregationType": "Maximum",
        "Cooldown": 60,
        "StepAdjustments":
          [
             {
               "MetricIntervalLowerBound": 0,
               "ScalingAdjustment": 1
             }
          ]
    },
)

{'PolicyARN': 'arn:aws:autoscaling:us-east-1:434444145045:scalingPolicy:bae2deb9-e21a-45c0-bd3c-f073cece1bd5:resource/sagemaker/inference-component/sagemaker-2025-03-01-11-30-13-897-component:policyName/Step-scaling-policy-sagemaker-2025-03-01-11-30-13-897-component',
 'Alarms': [],
 'ResponseMetadata': {'RequestId': 'abb43966-31a5-4acf-a255-dd8ea09f1711',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'abb43966-31a5-4acf-a255-dd8ea09f1711',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '279',
   'date': 'Sat, 01 Mar 2025 11:58:33 GMT'},
  'RetryAttempts': 0}}

In [87]:
resp = aas_client.describe_scaling_policies(
    PolicyNames=[step_scaling_policy_name,target_tracking_policy_name],
    ServiceNamespace=service_namespace,
    ResourceId=resource_id,
    ScalableDimension=scalable_dimension,
)
step_scaling_policy_arn = resp['ScalingPolicies'][0]['PolicyARN']
print(f"step_scaling_policy_arn: {step_scaling_policy_arn}")
print(resp['ScalingPolicies'])

step_scaling_policy_arn: arn:aws:autoscaling:us-east-1:434444145045:scalingPolicy:bae2deb9-e21a-45c0-bd3c-f073cece1bd5:resource/sagemaker/inference-component/sagemaker-2025-03-01-11-30-13-897-component:policyName/Step-scaling-policy-sagemaker-2025-03-01-11-30-13-897-component
[{'PolicyARN': 'arn:aws:autoscaling:us-east-1:434444145045:scalingPolicy:bae2deb9-e21a-45c0-bd3c-f073cece1bd5:resource/sagemaker/inference-component/sagemaker-2025-03-01-11-30-13-897-component:policyName/Step-scaling-policy-sagemaker-2025-03-01-11-30-13-897-component', 'PolicyName': 'Step-scaling-policy-sagemaker-2025-03-01-11-30-13-897-component', 'ServiceNamespace': 'sagemaker', 'ResourceId': 'inference-component/sagemaker-2025-03-01-11-30-13-897-component', 'ScalableDimension': 'sagemaker:inference-component:DesiredCopyCount', 'PolicyType': 'StepScaling', 'StepScalingPolicyConfiguration': {'AdjustmentType': 'ChangeInCapacity', 'StepAdjustments': [{'MetricIntervalLowerBound': 0.0, 'ScalingAdjustment': 1}], 'Cool

### Create the CloudWatch alarm that will trigger our policy
Finally, create a CloudWatch alarm with the metric NoCapacityInvocationFailures. When triggered, the alarm initiates the previously defined scaling policy. For more information about the NoCapacityInvocationFailures metric, see documentation.

We have also set the following:  

- EvaluationPeriods to 1  
- DatapointsToAlarm to 1  
- ComparisonOperator to GreaterThanOrEqualToThreshold  
- This results in 1 min waiting for the step scaling policy to trigger  



In [88]:
# The alarm name for the step scaling alarm
step_scaling_alarm_name = f"step-scaling-alarm-scale-to-zero-aas-{component_name}"

cloudwatch_client.put_metric_alarm(
    AlarmName=step_scaling_alarm_name,
    AlarmActions=[step_scaling_policy_arn],  # Replace with your actual ARN
    MetricName='NoCapacityInvocationFailures',
    Namespace='AWS/SageMaker',
    Statistic='Maximum',
    Dimensions=[
        {
            'Name': 'InferenceComponentName',
            'Value': component_name  # Replace with actual InferenceComponentName
        }
    ],
    Period=30, # 定义了 CloudWatch 收集和聚合指标数据的时间间隔，CloudWatch 支持的最小 Period 值通常为 10 或 60 秒，取决于指标类型和监控级别
    EvaluationPeriods=1, #定义了在多少个连续的数据点中需要满足条件才会触发警报，=1 表示只需要评估 1 个时间段（即 30 秒，由 Period 定义）
    DatapointsToAlarm=1, #表示在 EvaluationPeriods 中需要满足阈值条件的数据点数量，=1 表示在评估的 1 个时间段内，只要有 1 个数据点满足条件就触发警报
    Threshold=1, #表示当 NoCapacityInvocationFailures 指标值大于或等于 1 时触发警报
    ComparisonOperator='GreaterThanOrEqualToThreshold',
    TreatMissingData='missing' #缺失的数据点不会触发警报状态的任何变化，notBreaching将缺失的数据点视为"良好"或"未违反阈值"，breaching将缺失的数据点视为"违反阈值"，ignore保持当前警报状态不变，直到有新数据点出现
)

{'ResponseMetadata': {'RequestId': '7ba09c8e-b340-41a5-b9d7-8ea169bb5ac6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7ba09c8e-b340-41a5-b9d7-8ea169bb5ac6',
   'content-type': 'text/xml',
   'content-length': '214',
   'date': 'Sat, 01 Mar 2025 12:01:14 GMT'},
  'RetryAttempts': 0}}

### check updating status

In [96]:
while True:
    if  check_inference_component_status(component_name):
        break
    time.sleep(30)
start_time = time.time()

Inference Component Status: InService
Inference component has been successfully deployed


### Test the solution
Notice the MinInstanceCount: 0 setting in the Endpoint configuration, which allows the endpoint to scale down to zero instances. With the scaling policy, CloudWatch alarm, and minimum instances set to zero, your SageMaker Inference Endpoint will now be able to automatically scale down to zero instances when not in use, helping you optimize your costs and resource utilization.

### IC copy count scales in to zero
We'll pause for a few minutes without making any invocations to our model. Based on our target tracking policy, when our SageMaker endpoint doesn't receive requests for about 10 to 15 minutes, it will automatically scale down to zero the number of model copies.

In [98]:
import sys
# time.sleep(900)
while True:
    desc = sm_client.describe_inference_component(InferenceComponentName=component_name)
    status = desc["InferenceComponentStatus"]
    print(status)
    sys.stdout.flush()
    if status not in ["InService", "Failed"]:
        break
    time.sleep(30)

total_time = time.time() - start_time
print(f"\nTotal time taken: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")

desc = sm_client.describe_inference_component(InferenceComponentName=component_name)
print(desc)

InService


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:10                                                                                   │
│                                                                                                  │
│    7 │   sys.stdout.flush()                                                                      │
│    8 │   if status not in ["InService", "Failed"]:                                               │
│    9 │   │   break                                                                               │
│ ❱ 10 │   time.sleep(30)                                                                          │
│   11                                                                                             │
│   12 total_time = time.time() - start_time                                                       │
│   13 print(f"\nTotal time taken: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyboardInterrupt

### Endpoint's instances scale in to zero
After 10 additional minutes of inactivity, SageMaker automatically terminates all underlying instances of the endpoint, eliminating all associated costs.

In [52]:
# sess.wait_for_endpoint(endpoint_name)


- After 10 additional minutes of inactivity, SageMaker automatically stops all underlying instances of the endpoint, eliminating all associated instance costs.

If we try to invoke our endpoint while instances are scaled down to zero, we get a validation error:  
`An error occurred (ValidationError) when calling the InvokeEndpoint operation: Inference Component has no capacity to process this request. ApplicationAutoScaling may be in-progress (if configured) or try to increase the capacity by invoking UpdateInferenceComponentRuntimeConfig API.`

In [93]:
from sagemaker.predictor import retrieve_default 
from sagemaker import Predictor
from sagemaker import serializers, deserializers
# predictor = retrieve_default(endpoint_name, sagemaker_session=sess) 

predictor = Predictor(
            endpoint_name=endpoint_name,
            sagemaker_session=sess,
            serializer=serializers.JSONSerializer(),
            component_name=component_name
        )

payload = {
    "messages": [
    {
        "role": "user",
        "content": "who are you"
    }
    ],
    "model":"qwen",
    "max_tokens": 1024,
    "stream": False
}
response = predictor.predict(payload) 
print(response)

b'{"id":"1f6d3d67236b468a8ab992b576ef957c","object":"chat.completion","created":1740830510,"model":"qwen","choices":[{"index":0,"message":{"role":"assistant","content":"I\'m an artificial intelligence assistant designed to provide information, answer questions, and assist with various tasks for you. How can I help you today?","tool_calls":null},"logprobs":null,"finish_reason":"stop","matched_stop":151645}],"usage":{"prompt_tokens":22,"total_tokens":52,"completion_tokens":30,"prompt_tokens_details":null}}'


In [99]:
## test for scaling up
t1 = time.time()
i = 0
while True:
    predictor.predict(payload) 
    i += 1
    if i % 10 == 0:
        print(response)
        print(f"---{i}----time: {(time.time() - t1):.1f}")
    if i == 100000:
        break

b'{"id":"eea20689d118484b9d512b333fa44fe9","object":"chat.completion","created":1740832513,"model":"qwen","choices":[{"index":0,"message":{"role":"assistant","content":"I am an artificial intelligence designed to assist with answering questions, providing information, and engaging in conversation. I am here to help you with any queries you may have.","tool_calls":null},"logprobs":null,"finish_reason":"stop","matched_stop":151645}],"usage":{"prompt_tokens":22,"total_tokens":56,"completion_tokens":34,"prompt_tokens_details":null}}'
---10----time: 3.6
b'{"id":"eea20689d118484b9d512b333fa44fe9","object":"chat.completion","created":1740832513,"model":"qwen","choices":[{"index":0,"message":{"role":"assistant","content":"I am an artificial intelligence designed to assist with answering questions, providing information, and engaging in conversation. I am here to help you with any queries you may have.","tool_calls":null},"logprobs":null,"finish_reason":"stop","matched_stop":151645}],"usage":{"

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│    2 t1 = time.time()                                                                            │
│    3 i = 0                                                                                       │
│    4 while True:                                                                                 │
│ ❱  5 │   predictor.predict(payload)                                                              │
│    6 │   i += 1                                                                                  │
│    7 │   if i % 10 == 0:                                                                         │
│    8 │   │   print(response)                                                                     │
│                                                                                                  │
│ /home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/lib/python3.11/site-packages/sagemake │
│ r/base_predictor.py:212 in predict                                                               │
│                                                                                                  │
│   209 │   │   if inference_component_name:                                                       │
│   210 │   │   │   request_args["InferenceComponentName"] = inference_component_name              │
│   211 │   │                                                                                      │
│ ❱ 212 │   │   response = self.sagemaker_session.sagemaker_runtime_client.invoke_endpoint(**req   │
│   213 │   │   return self._handle_response(response)                                             │
│   214 │                                                                                          │
│   215 │   def _handle_response(self, response):                                                  │
│                                                                                                  │
│ /home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/lib/python3.11/site-packages/botocore │
│ /client.py:570 in _api_call                                                                      │
│                                                                                                  │
│    567 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    568 │   │   │   │   )                                                                         │
│    569 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  570 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    571 │   │                                                                                     │
│    572 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    573                                                                                           │
│                                                                                                  │
│ /home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/lib/python3.11/site-packages/botocore │
│ /context.py:124 in wrapper                                                                       │
│                                                                                                  │
│   121 │   │   │   with start_as_current_context():                                               │
│   122 │   │   │   │   if hook:                                                                   │
│   123 │   │   │   │   │   hook()                                                                 │
│ ❱ 124 │   │   │   │   return func(*args, **kwargs)         

### 4.4 Delete

In [100]:
!aws sagemaker delete-inference-component --inference-component-name {component_name}


In [101]:
!aws sagemaker delete-endpoint --endpoint-name {endpoint_name}


In [102]:
!aws sagemaker delete-endpoint-config --endpoint-config-name {endpoint_config_name}


In [103]:
!aws sagemaker delete-model --model-name {model_name}